In [ ]:
import timeit
import os 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import json
import timm

from numpy import expand_dims
from pytorch_lightning import Trainer
from matplotlib import pyplot
from PIL import Image
from torch.utils.data import DataLoader

from keras.preprocessing.image import ImageDataGenerator  
from keras.utils import load_img, img_to_array
from anomalib.models import get_model
from anomalib.models.padim.lightning_model import Padim
from anomalib.config import get_configurable_parameters
from anomalib.utils.callbacks import LoadModelCallback, get_callbacks
from anomalib.data.folder import Folder
from anomalib.data.mvtec import MVTec

from anomalib.models.fastflow.lightning_model import Fastflow
from anomalib.models.dfm.lightning_model import Dfm
from anomalib.models.patchcore.lightning_model import Patchcore
from anomalib.models.cflow.lightning_model import Cflow

In [ ]:
def anomaset(model, tuned_model):
    CONFIG_PATH = f"./anomalib/models/{model}/config.yaml"

    config = get_configurable_parameters(config_path=CONFIG_PATH)

    datamodule = MVTec(
        root="../data/mvtec_anomaly_detection",
        category='capsule',
        image_size=256,
        train_batch_size=32,
        test_batch_size=32,
        num_workers=8,
        task ="segmentation",
        seed = 42)

    datamodule.setup()
    model = tuned_model

    callbacks = get_callbacks(config)

    return datamodule, model, callbacks, config

def anomatrain(datamodule, model, callbacks, config):
    trainer = Trainer(**config.trainer, callbacks=callbacks)
    trainer.fit(model=model, datamodule=datamodule)
    load_model_callback = LoadModelCallback(weights_path=trainer.checkpoint_callback.best_model_path)
    trainer.callbacks.insert(0, load_model_callback)
    return trainer

def anomatest(trainer, model, datamodule):
    trainer.test(model=model, datamodule=datamodule)
    df = trainer.logged_metrics
    return  df

def anomastudy(model):
    datamodule, model, callbacks, config = anomaset(model)
    trainer = anomatrain(datamodule, model, callbacks, config)
    metrics = anomatest(trainer, model, datamodule)
    return metrics

def anomaresults(models):
    for model in models:
        metrics = anomastudy(model)
    return metrics

# Data augmentation

In [ ]:
def img_resize(path, foldertype, newfolder):
    imgs = os.listdir(path)
    for img in imgs:
        img_arr = np.array(Image.open(path+'/'+img)) # (288, 352, 3)  
        img_new = cv2.resize(img_arr, (288,288))
        im = Image.fromarray(img_new)
        im.save(f'./data/{newfolder}/{foldertype}/{img}')

def resize_img_pipe(oldfolder, newfolder):
    os.mkdir(f'./data/{newfolder}')
    os.mkdir(f'./data/{newfolder}/abnormal')
    os.mkdir(f'./data/{newfolder}/normal')

    img_resize(f'./data/{oldfolder}/normal/','normal',newfolder)
    img_resize(f'./data/{oldfolder}/abnormal/','abnormal', newfolder)

def image_augmenter(path, folder, n, newfolder):
    img = img_to_array(load_img(path))
    data = expand_dims(img, 0)
    datagen = ImageDataGenerator(#rotation_range=40,
                                    width_shift_range=0.2, 
                                    #featurewise_center=True,
                                    #featurewise_std_normalization=True,
                                    height_shift_range=0.2, 
                                    #shear_range=0.2, 
                                    #zoom_range=0.2,
                                    #rotation_range=90,
                                    brightness_range=[0.4,1.3],
                                    horizontal_flip=True,
                                    #zca_whitening=True,
                                    vertical_flip=True,
                                    fill_mode='nearest')
    count = 0                                
    for batch in datagen.flow(data, batch_size=1, save_prefix='orange', save_to_dir=f'./data/{newfolder}/{folder}',save_format='jpg'):
        count += 1
        if count == n:
            break
     
def gen_img_pipe(oldfolder, newfolder, n, generate):
    resize_img_pipe(oldfolder, newfolder)
    if generate:
        normal = os.listdir(f'./data/{newfolder}/normal')
        abnormal =  os.listdir(f'./data/{newfolder}/abnormal')
        for end in normal:
            image_augmenter(f'./data/{newfolder}/normal/{end}','normal', n, newfolder)
        for end in abnormal:
            image_augmenter(f'./data/{newfolder}/abnormal/{end}', 'abnormal', n, newfolder)

In [ ]:
gen_img_pipe('naranjas_pynq_0','naranjas_pynq_2_crop_augmentation', 10, True)

# Transfer Learning

In [ ]:
model_names = timm.list_models()
model_names

In [ ]:
model = timm.create_model('efficientnet_l2', pretrained=True)
#print(model_names)
layer_names = [name for name, _ in model.named_children()]
print(layer_names)

In [ ]:
models = ['padim']#,'draem','dfm', 'cflow', 'stfpm', 'ganomaly', 'dfkde', 'patchcore']
# categories = ['bottle', 'cable', 'capsule', 'carpet', 'grid', 'hazelnut', 'leather', 'metal_nut', 'pill', 'screw', 'tile', 'toothbrush', 'transistor', 'wood', 'zipper']
new_model = Dfm(backbone='efficientnet_l2',layer='conv_head')

metrics = anomaresults(models, new_model)
metrics

# Hyperparameter tuning

In [ ]:
help(Dfm)

In [ ]:
models = ['dfm']
tuned_model = Dfm(backbone='efficientnet_l2',layer='conv_head',score_type='fre', pooling_kernel_size=4)
metrics = anomaresults(models, tuned_model)
metrics

In [ ]:
tuned_model_1 = Dfm(backbone='efficientnet_l2',layer='conv_head',score_type='nll', pooling_kernel_size=2)
tuned_model_2 = Patchcore(input_size=[256, 256], backbone="resnet18", layers=['layer2','layer3'],coreset_sampling_ratio=0.3, num_neighbors=18)
tuned_model_3 = Cflow(input_size=[256, 256], backbone="resnet18", layers=['layer2','layer3','layer4'])
tuned_model_4 = Fastflow(input_size=[256,256], backbone='resnet18',weight_decay=0.0001)

 # WandB

In [ ]:
hpo_yaml_file = """
observation_budget: 10
method: bayes
metric:
  name: pixel_AUPR
  goal: maximize
parameters:
  model:
    pooling_kernel_size:
      values: [2,4]
"""

with open("C:/Users/Pablo/Desktop/hpo.yaml", "w", encoding="utf8") as f:
    f.writelines(hpo_yaml_file)

In [ ]:
!python "C:/Users/Pablo/anomalib/tools/hpo/sweep.py" --model dfm --model_config "C:/Users/Pablo/anomalib/anomalib/models/dfm/config.yaml" --sweep_config "C:/Users/Pablo/Desktop/hpo.yaml"